# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

## 1. Setup and Imports

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


## 2. Build a Technical Chat Assistant for GenAI queries 

In [5]:
system_prompt = "You are an assistant that responds to technical question mostly related to generative AI and large language models. \
Try and understand the user experience first by having a conversation with the user before you answer their question. \
answer their questions considering their level of experience and helping grow their knowledge of the topic.\
Include details which can explain the response and teaches the reader about the topic. \
be factual and if you don't know the answer simply say you don't know."

In [6]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [7]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


## 3. Add Tool functionality

In [8]:
# Let's start by making a useful function

model_links = {"gpt": "https://aws.amazon.com/what-is/gpt/?utm_source=chatgpt.com", 
               "sonet3.5": "https://www.anthropic.com/news/claude-3-5-sonnet?utm_source=chatgpt.com",
               "deepseekr1": "https://www.vellum.ai/blog/the-training-of-deepseek-r1-and-ways-to-use-it"}

def get_model_links(model):
    print(f"Tool get_model_links called for {model}")
    link = model.lower()
    return model_links.get(model, "Unknown")

In [9]:
get_model_links('gpt')

Tool get_model_links called for gpt


'https://aws.amazon.com/what-is/gpt/?utm_source=chatgpt.com'

In [10]:
# There's a particular dictionary structure that's required to describe our function:

model_function = {
    "name": "get_model_links",
    "description": "Get the related links that describes a model. Call this whenever you need to provide the user links to learn more about a model, for example when a user asks 'How can I learn more about gpt'",
    "parameters": {
        "type": "object",
        "properties": {
            "model": {
                "type": "string",
                "description": "The model that the user wants to learn about",
            },
        },
        "required": ["model"],
        "additionalProperties": False
    }
}

In [11]:
# And this is included in a list of tools:
tools = [{"type": "function", "function": model_function}]

In [13]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, model = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [14]:
# We have to write that function handle_tool_call:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    model = arguments.get('model')
    link = get_model_links(model)
    response = {
        "role": "tool",
        "content": json.dumps({"model": model,"link": link}),
        "tool_call_id": tool_call.id
    }
    return response, model

In [15]:
gr.ChatInterface(fn=chat, type="messages", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


Tool get_model_links called for sonet3.5
